In [438]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime as dt
import copy
from sqlalchemy import create_engine, MetaData

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

##### Import Data

In [439]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("data/t2m_classification.xlsx", sheet_name='name_map').drop(columns=['group'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

In [440]:
#Đọc toàn bộ các file csv được xuất ra từ ami eod
eod_item_dict = {}
folder_path = '../ami_eod_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        eod_item_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

for item, df in eod_item_dict.items():
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%y%m%d')
    eod_item_dict[item] = df

#Tạo bảng tổng hớp tất cả các item
eod_item_df = pd.DataFrame(list(eod_item_dict.keys())).rename(columns={0:'item'})
eod_item_df['len'] = eod_item_df['item'].apply(lambda x: len(x))
eod_item_df['last_2chars'] = eod_item_df['item'].str[-2:]
eod_item_df['first_4chars'] = eod_item_df['item'].str[:4]

#Lọc ra danh sách tên các cổ phiếu và index
stock_name_df = eod_item_df[eod_item_df['len']==3].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)
index_name_df = eod_item_df[(eod_item_df['len']>3) & (eod_item_df['len']!=6) & (eod_item_df['len']<10) & 
                (eod_item_df['item']!='0001')].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)
                
eod_stock_dict = {k:v.drop(['option'], axis=1) for k,v in eod_item_dict.items() if k in stock_name_df['item'].tolist()}
eod_index_dict = {k:v.rename(columns={'option':'value'}).drop('cap', axis=1)
                for k,v in eod_item_dict.items() if k in index_name_df['item'].tolist()}

#Lọc ra danh sách tên các cổ phiếu, index giao dịch tự doanh và nước ngoài
stock_td_nn_df = eod_item_df[(eod_item_df['len']==6) & (eod_item_df['last_2chars'].isin(['TD','NN']))].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)
index_td_nn_df = eod_item_df[(eod_item_df['len']>=10) & (eod_item_df['first_4chars']!='VN30') & ((eod_item_df['last_2chars']=='NN') | (eod_item_df['last_2chars']=='TD'))].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)

stock_td_nn_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in stock_td_nn_df['item'].tolist()}
index_td_nn_dict = {k:v.drop(['high','low','cap','stock'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in index_td_nn_df['item'].tolist()}

#Điều chỉnh đơn vị của các bảng NN và TD
for df in index_td_nn_dict.values():
    df['buy_volume'] = df['buy_volume']/1000
    df['sell_volume'] = -df['sell_volume']/1000
    df['buy_value'] = df['buy_value']/1000000000
    df['sell_value'] = -df['sell_value']/1000000000
    df['net_volume'] = df['buy_volume'] + df['sell_volume']
    df['net_value'] = df['buy_value'] + df['sell_value']

In [441]:
#Tạo một date_series bao gồm khoảng ngày tính toán eod
date_series = pd.DataFrame(eod_index_dict['VNINDEX']['date']).rename(columns={0:'date'})

#Tạo một time_series bao gồm khoảng ngày tính toán itd
time_series_list = []
for day in date_series['date'].iloc[:6].tolist():
    time_series_list.extend(pd.date_range(start=f'{day} 09:00:00', end=f'{day} 11:25:00', freq='5T'))
    time_series_list.extend(pd.date_range(start=f'{day} 13:00:00', end=f'{day} 15:00:00', freq='5T'))
time_series = pd.DataFrame(time_series_list).rename(columns={0:'date'})

In [475]:
#Đọc toàn bộ các file csv được xuất ra từ ami itd
itd_item_dict = {}
folder_path = '../ami_itd_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        itd_item_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

current_time = pd.to_datetime(itd_item_dict['ACV']['date'].iloc[0], format='%y%m%d %H%M%S')

for item, df in itd_item_dict.items():
    #Chuyển đổi về dạng m5
    df['minutes'] = df['date'].str.replace(' ', '').str.slice(4, 10).apply(lambda x: (x[:-1] + '0') if int(x[-1]) < 5 else (x[:-1] + '5'))
    df['date'] = df['date'].str.slice(0,7) + df['minutes'].str.slice(2) + "00"

    df = df.groupby(df['date']).agg({
            'stock': 'first',
            'date': 'last',
            'open': 'last',
            'high': 'max',
            'low': 'min',
            'close': 'first',
            'volume': 'sum'
        }).reset_index(drop=True)
    
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%y%m%d %H%M%S')
    df = time_series.merge(df, on='date', how='left').sort_values('date', ascending=False)
    df[['stock','open','high','low','close']] = df[['stock','open','high','low','close']].fillna(method='bfill')
    df['volume'] = df['volume'].fillna(0)
    df = df.loc[df['date'] > date_series['date'].iloc[4]].reset_index(drop=True)

    #Xoá những giá trị bị fill quá thời gian hiện tại:
    df.loc[df['date'] > current_time, ['open','high','low','close']] = None
    df = df.dropna()

    itd_item_dict[item] = df

#Tạo bảng tổng hợp tất cả các item
itd_item_df = pd.DataFrame(list(itd_item_dict.keys())).rename(columns={0:'item'})
itd_item_df['len'] = itd_item_df['item'].apply(lambda x: len(x))
itd_item_df['last_2chars'] = itd_item_df['item'].str[-2:]
itd_item_df['third_last_char'] = itd_item_df['item'].str[-3:-2]
itd_item_df['first_4chars'] = itd_item_df['item'].str[:4]

#Lọc ra danh sách tên các cổ phiếu và index
stock_name_df = itd_item_df[itd_item_df['len']==3].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_name_df = itd_item_df[(itd_item_df['len']>3) & (itd_item_df['len']!=6) & (itd_item_df['len']<10) & (itd_item_df['item']!='0001')]\
                .reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)

itd_stock_dict = {k:v for k,v in itd_item_dict.items() if k in stock_name_df['item'].tolist()}
itd_index_dict = {k:v.rename(columns={'option':'value'})
                for k,v in itd_item_dict.items() if k in index_name_df['item'].tolist()}

In [476]:
#Xoá các cổ phiếu chưa có giao dịch trong ngày
delete_stock = []
for stock, df in eod_stock_dict.items():
    if date_series['date'].iloc[0] != df['date'].iloc[0]:
        delete_stock.append(stock)
for stock in delete_stock:
    itd_stock_dict.pop(stock)
    eod_stock_dict.pop(stock)

#Xoá các cổ phiếu có giá bị lỗi bằng 0
delete_stock = []
for stock, df in eod_stock_dict.items():
    if df['close'].min() == 0:
        delete_stock.append(stock)
for stock in delete_stock:
    itd_stock_dict.pop(stock)
    eod_stock_dict.pop(stock)

#Đổi lại cap của cổ phiếu thành cap trung bình trong 20 phiên
for df in eod_stock_dict.values():
    df['cap'] = df['cap'][::-1].rolling(window=20).mean()[::-1]

In [477]:
#Tính toán các đường trung bình và các đường MA
eod_stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in eod_stock_dict.items()}

eod_stock_dict = {
    key: df.assign(
        high5=df['high'].rolling(window=5, min_periods=1).max(),
        low5=df['low'].rolling(window=5, min_periods=1).min(),
        high20=df['high'].rolling(window=20, min_periods=1).max(),
        low20=df['low'].rolling(window=20, min_periods=1).min(),
        high60=df['high'].rolling(window=60, min_periods=1).max(),
        low60=df['low'].rolling(window=60, min_periods=1).min(),
        high120=df['high'].rolling(window=120, min_periods=1).max(),
        low120=df['low'].rolling(window=120, min_periods=1).min(),
        high240=df['high'].rolling(window=240, min_periods=1).max(),
        low240=df['low'].rolling(window=240, min_periods=1).min(),
        high480=df['high'].rolling(window=480, min_periods=1).max(),
        low480=df['low'].rolling(window=480, min_periods=1).min(),

        ma5_V=df['volume'].rolling(window=5, min_periods=1).mean().shift(1),
        ma20_V=df['volume'].rolling(window=20, min_periods=1).mean().shift(1),
        ma60_V=df['volume'].rolling(window=60, min_periods=1).mean().shift(1),
        ma120_V=df['volume'].rolling(window=120, min_periods=1).mean().shift(1),

        ma5=df['close'].rolling(window=5, min_periods=1).mean(),
        ma20=df['close'].rolling(window=20, min_periods=1).mean(),
        ma60=df['close'].rolling(window=60, min_periods=1).mean(),
        ma120=df['close'].rolling(window=120, min_periods=1).mean(),
        ma240=df['close'].rolling(window=240, min_periods=1).mean(),
        ma480=df['close'].rolling(window=480, min_periods=1).mean(),
    )
    for key, df in eod_stock_dict.items()
}

eod_stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int)
    )
    for key, df in eod_stock_dict.items()
}
eod_stock_dict = {k: v.sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in eod_stock_dict.items()}

In [478]:
#Gán các đường trung bình và MA sang bảng dữ liệu ITD
for stock, df in itd_stock_dict.items():
    temp_data = eod_stock_dict[stock][['high5', 'low5', 'high20', 'low20', 'high60', 'low60',
                                       'high120','low120', 'high240', 'low240', 'high480', 'low480']].iloc[0]
    itd_stock_dict[stock] = df.assign(**temp_data)


itd_stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in itd_stock_dict.items()}
itd_stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int)
    )
    for key, df in itd_stock_dict.items()
}
itd_stock_dict = {k: v.sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in itd_stock_dict.items()}

##### Phân nhóm cổ phiếu

In [480]:
stock_classification = pd.read_excel('data/t2m_classification.xlsx')
stock_classification = stock_classification[stock_classification['stock'].isin(list(eod_stock_dict.keys()))]

#Tạo ngày đầu tiên của tháng hiện tại
first_day_of_month = date_series[date_series['date'] > pd.Timestamp(df['date'].iloc[0].year, df['date'].iloc[0].month, 1)]['date'].iloc[-1]

#Tạo các mảng dữ liệu vốn hoá và giá của phiên đầu tiên hàng tháng
price_arr = []
cap_arr = []
for stock, df in eod_stock_dict.items():
    price_arr.append(df[df['date'] == first_day_of_month]['close'].item())
    cap_arr.append(df[df['date'] == first_day_of_month]['cap'].iloc[0].item())

#Tạo bảng chia nhóm vốn hoá
vonhoa_classification_df = stock_classification.copy()
vonhoa_classification_df['price'] = price_arr
vonhoa_classification_df['cap'] = cap_arr

cap_coef = sum(cap_arr)/10000
vonhoa_classification_df['marketcap_group'] = vonhoa_classification_df.apply(lambda x:
    'small' if ((x['cap']>cap_coef) & (x['cap']<10*cap_coef)) | 
               ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']<10)) 
               else
    ('mid' if ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']>=10)) | 
              ((x['cap']>=20*cap_coef) & (x['cap']<100*cap_coef))
              else
    ('large' if x['cap']>=100*cap_coef
               else 'penny'
)), axis=1)

stock_classification = pd.concat([stock_classification, vonhoa_classification_df['marketcap_group']], axis=1)

In [481]:
eod_all_stock = {}
eod_all_stock['all_stock'] = {key: value for key, value in eod_stock_dict.items()}

eod_industry_name = {}
for name in np.sort(stock_classification['industry_name'].unique()):
    eod_industry_name[name] = {key: value for key, value in eod_stock_dict.items() if key in stock_classification[stock_classification['industry_name'] == name]['stock'].to_list()}

eod_industry_perform = {}
for name in np.sort(stock_classification['industry_perform'].unique()):
    eod_industry_perform[name] = {key: value for key, value in eod_stock_dict.items() if key in stock_classification[stock_classification['industry_perform'] == name]['stock'].to_list()}

eod_marketcap_group = {}
for name in ['large', 'mid', 'small', 'penny']:
    eod_marketcap_group[name] = {key: value for key, value in eod_stock_dict.items() if key in stock_classification[stock_classification['marketcap_group'] == name]['stock'].to_list()}

In [482]:
itd_all_stock = {}
itd_all_stock['all_stock'] = {key: value for key, value in itd_stock_dict.items()}

itd_industry_name = {}
for name in np.sort(stock_classification['industry_name'].unique()):
    itd_industry_name[name] = {key: value for key, value in itd_stock_dict.items() if key in stock_classification[stock_classification['industry_name'] == name]['stock'].to_list()}

itd_industry_perform = {}
for name in np.sort(stock_classification['industry_perform'].unique()):
    itd_industry_perform[name] = {key: value for key, value in itd_stock_dict.items() if key in stock_classification[stock_classification['industry_perform'] == name]['stock'].to_list()}

itd_marketcap_group = {}
for name in ['large', 'mid', 'small', 'penny']:
    itd_marketcap_group[name] = {key: value for key, value in itd_stock_dict.items() if key in stock_classification[stock_classification['marketcap_group'] == name]['stock'].to_list()}

##### Biểu đồ cấu trúc sóng

In [492]:
def transform_ms(stock_group, time_interval):
    stock_dict = copy.deepcopy(stock_group)

    if time_interval == 'eod':
        temp_time_series = copy.deepcopy(date_series)
    elif time_interval == 'itd':
        temp_time_series = copy.deepcopy(time_series)

    for group_name in stock_dict.keys():

        #Tạo dict chứa các cổ phiếu thuộc cùng nhóm
        trend_dict = {'trend_5p': None, 'trend_20p': None,'trend_60p': None,'trend_120p': None,'trend_240p': None, 'trend_480p': None}
        for trend in trend_dict.keys():
            trend_dict[trend] = pd.DataFrame(temp_time_series['date'].tolist(), columns=['date'])
            for stock in stock_dict[group_name].keys():
                trend_dict[trend][stock] = stock_dict[group_name][stock][trend]
            trend_dict[trend].fillna(value=0, inplace=True)
            trend_dict[trend]['sum'] = trend_dict[trend].iloc[:, 1:len(stock_dict[group_name])+1].apply(sum, axis=1)
            trend_dict[trend]['percent'] = trend_dict[trend]['sum'] / len(stock_dict[group_name])

        #Tạo bảng dữ liệu theo ngày
        stock_dict[group_name] = pd.DataFrame(temp_time_series['date'].tolist(), columns=['date']).sort_values('date', ascending=False)
        for trend in trend_dict.keys():
            stock_dict[group_name][trend] = trend_dict[trend]['percent']

    return stock_dict

In [493]:
eod_all_stock_ms = transform_ms(eod_all_stock, 'eod')
eod_industry_name_ms = transform_ms(eod_industry_name, 'eod')
eod_industry_perform_ms = transform_ms(eod_industry_perform, 'eod')
eod_marketcap_group_ms = transform_ms(eod_marketcap_group, 'eod')

In [497]:
itd_all_stock_ms = transform_ms(itd_all_stock, 'itd')

In [498]:
itd_all_stock_ms['all_stock']

,date,trend_5p,trend_20p,trend_60p,trend_120p,trend_240p,trend_480p
54,2024-04-10 15:00:00,0.282908,0.259332,0.467583,0.636542,0.473477,0.475442
53,2024-04-10 14:55:00,0.332024,0.290766,0.479371,0.636542,0.483301,0.475442
52,2024-04-10 14:50:00,0.341847,0.294695,0.479371,0.640472,0.483301,0.475442
51,2024-04-10 14:45:00,0.355599,0.298625,0.479371,0.642436,0.483301,0.475442
50,2024-04-10 14:40:00,0.353635,0.296660,0.481336,0.642436,0.483301,0.475442
...,...,...,...,...,...,...,...
279,2024-04-03 09:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
278,2024-04-03 09:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
277,2024-04-03 09:10:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
276,2024-04-03 09:05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
